<a href="https://colab.research.google.com/github/Seb207/Market-Context-Project/blob/main/Data%20Preprocessing/Model_Analysis_Lab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Financial Analysis Tools Used To Find Meaningful Insight

# List of tools
- Performance & Risk (shape ratio, rolling return, MDD, etc.)
- Technical Analysis
- Fundamental Analysis

**List of Libraries**
- Pyfolio --> performance stats
- Zipline --> performance stats, backtest
- Quantlib -->

In [ ]:
pip install pyfolio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.1/91.1 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 28.1 MB/s eta 0:00:00
  Created wheel for pyfolio: filename=pyfolio-0.9.2-py3-none-any.whl size=88654 sha256=339300ba1384dfedc6c7d1a2bbb2bb40f0ac1db6942cf0068a5ab4c394d68992
  Stored in directory: /root/.cache/pip/wheels/f9/af/9e/7c343b822164a3147a3d395a1bcd05041c520a3bc6398fe88e
  Created wheel for empyrical: filename=empyrical-0.5.5-py3-none-any.whl size=39752 sha256=16546daabf6e5d2c7284b3c08e786926398c622051e7c62313646b0134d1d031
  Stored in directory: /root/.cache/pip/wheels/ac/1d/58/a7ae5ef5c8de7c4b769f24c2584f4706564921f031b16b9cb6
Successfully built pyfolio empyrical


In [ ]:
pip install zipline

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 25.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 45.1 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × pip subprocess to install build dependencies did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Installing build dependencies ... error
error: subprocess-exited-with-error

× pip subprocess to install build dependencies did not run successfully.
│ exit code: 1
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.


In [ ]:
import pandas as pd
import numpy as np

import pyfolio as pf

/usr/local/lib/python3.11/dist-packages/pyfolio/pos.py:26: UserWarning: Module "zipline.assets" not found; mutltipliers will not be applied to position notionals.
  warnings.warn(


##1. Performance & Risk

- Data is preprocessed (start-end, dropna) before passing through the formula

#1.1 Sharpe Ratio

Rate of return corresponding to the risk

Parameters:
- Expected portfolio return (RE)
- Risk free rate of return (RF)
- STD of portfolio return (volatility) (V)

Formula:
(RE - RF) / V

Purpose:
- Maximizing return and reducing volatility of the entire **portfolio**

In [ ]:
def sharpe_ratio(self, dataset : pd.Series, rate : float) -> float:
  """
  sharpe ratio

  parameters
  ------------------
  dataset: price data
  rate: risk free rate of return (annual rate)
  annualized: Boolean whether the result has to be annualized

  returns
  ------------------
  sharpe_ratio
  """

  ret = dataset.pct_change()              # return for each trading day (s[i] / s[i-1])
  cum_ret = (1 + ret).prod()              # cummulative return of the entire period (add 1 for each element to make it growing)
  periods = ret.shape[0]                  # length of the series --> period
  ann_ret = cum_ret ** (252/periods) - 1  # Assume that the market is open 252 days per year
  volatility = ret.std()                  # volatility is set as the standard deviation of the return
  ann_vol = volatility * (252**0.5)       # annual volatility --> annual_std = (252 * variance) ** 0.5

  sharpe_ratio = (ann_ret - rate) / ann_vol

  return sharpe_ratio

#1.2 Sortino ratio

Rate of return corresponding to the downside volatility

Parameters:
- Expected portfolio return (RE)
- Risk free rate of return (RF)
- Downside STD of portfolio return (volatility) (V[-])

Formula:
(RE - RF) / V[-]

Purpose:
- Similar as the sharpe ratio, however, only considers downside risk

In [ ]:
def sortino_ratio(self, dataset : pd.Series, rate : float):
  """
  rate of return corresponding to the downside volatility

  parameters
  ----------
  dataset: time series price data of the target
  rate: risk-free rate of return

  returns
  -------
  sortino-ratio
  """

  ret = dataset.pct_change()              # return for each trading day (s[i] / s[i-1])
  cum_ret = (1 + ret).prod()              # cummulative return of the entire period (add 1 for each element to make it growing)
  periods = ret.shape[0]                  # length of the series --> period
  ann_ret = cum_ret ** (252/periods) - 1  # Assume that the market is open 252 days per year
  negative_ret = ret[ret < 0]             # Filters negative returns from "ret"
  volatility = negative_ret.std()         # volatility is set as the standard deviation of the return
  ann_vol = volatility * (252**0.5)       # annual volatility --> annual_std = (252 * variance) ** 0.5

  if ann_vol == 0:
    return np.nan

  sortino_ratio = (ann_ret - rate) / ann_vol

  return sortino_ratio

#1.3 Maximum Draw Down

In [ ]:
def maximum_drawdown (self, dataset : pd.Series):
  """
  parameters
  ----------
  dataset: time series data with (date vs. EOD price)

  returns
  -------
  -maximum_drawdown: MDD of the input data (negative value)
  dd_max: MDD max price
  dd_min: MDD min price

  """

  max_drawdown = 0
  max_price = 0
  dd_max = 0
  dd_min = 0
  num_days = len(dataset)

  for i in range(0, num_days):
    max_price = max(max_price, dataset[i])
    drawdown = (max_price - dataset[i]) / max_price

    if drawdown > max_drawdown:
      max_drawdown = drawdown
      dd_max = max_price
      dd_min = dataset[i]

  maximum_drawdown = - maximum_drawdown

  return (max_drawdown, dd_max, dd_min)


In [ ]:
def maximum_drawdown_simple (self, dataset : pd.Series):
  rets = dataset.pct_change()
  cumm_rets = (1 + rets).cumprod()

  peak = cumm_rets.cummax()
  drawdown = (cumm_rets - peak) / peak
  maximum_drawdown = drawdown.min()

  return maximum_drawdown

#1.4 Calmar Ratio

In [ ]:
def calmar_ratio (self, dataset : pd.Series):
  """
  parameters
  ----------
  dataset: time series data with (date vs. EOD price)

  returns
  -------
  calmar_ratio:

  """

  max_drawdown = maximum_drawdown(dataset)[0]

  if max_drawdown == 0:
    return np.nan

  ret = dataset.pct_change()              # return for each trading day (s[i] / s[i-1])
  cum_ret = (1 + ret).prod()              # cummulative return of the entire period (add 1 for each element to make it growing)
  periods = ret.shape[0]                  # length of the series --> period
  ann_ret = cum_ret ** (252/periods) - 1  # Assume that the market is open 252 days per year

  calmar_ratio = ann_ret / abs(max_drawdown)

  return calmar_ratio

#1.5 Markowitz (MPT)

#1.6 Kelley Criterion

##2. Technical Analysis

- Moving Average
- RSI
- Bollinger Band
- MACD
- ARIMA
- GARCH

#2.1 Moving Average
- Simple Moving Average
- EWMA

##3. Fundamental Analysis

- CAPM
- VaR, CVaR
- PER
- FWD PER
- PBR
- ROE

##4. Optimization

- Gradient Descent
- Global & Local Min

##5. Stochastic Process & Simulation

- Random Variable
- Monte Carlo Simulation
- BSM
- Heston Model
- Jump Diffusion
- LSM
- Valuation & Risk Measure